# 문맥 검색을 위한 검색 구현

KNN, BM25, Rank Fusion 등 다양한 검색 전략을 구현하고 테스트하는 방법을 안내해드리도록 하겠습니다.

In [ ]:
import os
import json
import yaml
import boto3
from dotenv import load_dotenv
from libs.rag_utils import OpenSearch_Manager

load_dotenv()

## 서비스 초기화하기

In [ ]:
# OpenSearch Manager 초기화
os_manager = OpenSearch_Manager()

# 모델 설정 불러오기
with open("libs/config.yml", "r") as file:
    model_config = yaml.safe_load(file)

# Bedrock 클라이언트 설정
bedrock_client = boto3.client('bedrock-runtime', region_name='us-west-2')

# 임베딩 모델 가져오기
embed_model = list(model_config['embedding_models'].keys())[0]
embed_model_id = model_config['embedding_models'][embed_model]['model_id']

## 다양한 검색 방법 테스트하기

In [ ]:
def get_embedding(text):
    response = bedrock_client.invoke_model(
        modelId=embed_model_id,
        body=json.dumps({"inputText": text})
    )
    return json.loads(response['body'].read())['embedding']

# 테스트 쿼리
query = "Amazon Bedrock의 주요 기능은 무엇인가요?"
index_name = "aws_contextual_test"  # 사용하실 인덱스명으로 변경해주세요

# 쿼리 임베딩 생성
query_embedding = get_embedding(query)

### KNN 검색 테스트

In [ ]:
knn_results = os_manager.search_by_knn(
    vector=query_embedding,
    index_name=index_name,
    top_n=5
)

print("\nKNN 검색 결과:")
for i, result in enumerate(knn_results, 1):
    print(f"\n결과 {i} (점수: {result['score']})")
    print(f"내용: {result['content'][:200]}...")

### BM25 검색 테스트

In [ ]:
bm25_results = os_manager.search_by_bm25(
    query_text=query,
    index_name=index_name,
    top_n=5
)

print("\nBM25 검색 결과:")
for i, result in enumerate(bm25_results, 1):
    print(f"\n결과 {i} (점수: {result['score']})")
    print(f"내용: {result['content'][:200]}...")

### Rank Fusion 테스트

In [ ]:
fusion_results = os_manager.search_by_rank_fusion(
    query_text=query,
    vector=query_embedding,
    index_name=index_name,
    initial_search_results=160,
    hybrid_score_filter=40,
    final_reranked_results=5,
    knn_weight=0.6
)

print("\nRank Fusion 결과:")
for i, result in enumerate(fusion_results, 1):
    print(f"\n결과 {i}")
    print(f"재순위화 점수: {result['score']}")
    print(f"하이브리드 점수: {result['hybrid_score']}")
    print(f"검색 방법: {result['search_methods']}")
    print(f"내용: {result['content'][:200]}...")

## 결과 비교하기

In [ ]:
def compare_results(knn, bm25, fusion):
    print("검색 결과 중복 분석:")
    knn_contents = set(r['content'] for r in knn)
    bm25_contents = set(r['content'] for r in bm25)
    fusion_contents = set(r['content'] for r in fusion)
    
    print(f"\nKNN 고유 결과: {len(knn_contents - (bm25_contents | fusion_contents))}")
    print(f"BM25 고유 결과: {len(bm25_contents - (knn_contents | fusion_contents))}")
    print(f"Fusion 고유 결과: {len(fusion_contents - (knn_contents | bm25_contents))}")
    
    print(f"\n모든 방법에서 공통된 결과: {len(knn_contents & bm25_contents & fusion_contents)}")
    print(f"KNN과 BM25에서만 공통된 결과: {len(knn_contents & bm25_contents - fusion_contents)}")
    print(f"KNN과 Fusion에서만 공통된 결과: {len(knn_contents & fusion_contents - bm25_contents)}")
    print(f"BM25와 Fusion에서만 공통된 결과: {len(bm25_contents & fusion_contents - knn_contents)}")

compare_results(knn_results, bm25_results, fusion_results)